In [99]:
import requests
import json
import pandas as pd
import itertools

In [56]:
config = pd.read_csv('google.config')

In [57]:
# Example usage
api_key = config.key[0]
cx_key = config.cx[0]

In [77]:
def search_companies(query, api_key):
    
    # API endpoint
    url = 'https://places.googleapis.com/v1/places:searchText'
    
    # Data payload
    payload = {
      "textQuery": query
    }
    
    # Headers
    headers = {
      'Content-Type': 'application/json',
      'X-Goog-Api-Key': api_key,
      'X-Goog-FieldMask': 'places.displayName,places.formattedAddress,places.websiteUri'
    }
    
    # Make the POST request
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    
    return response

In [100]:
types = ['Technology', 
         'Tech', 
         'AI', 
         'Artificial Intelligence',
         'Machine Learning',
         'ML',
         'Data Science',
         'Data Analytics',
         'Green',
         'Climate Change',
         'Global Warming',
         'Venture Backed',
         'Seed Funded',
         'Series A Funded',
         'Series B Funded',
         'Series C Funded',
         'Series D Funded'
        ]

where = ['Philadelphia, PA',
         'Greater Philadelphia Region',
         'Havertown, PA',
         'Mainline PA']

queries = [f'{type} Companies in {loc}' for type, loc in itertools.product(types, where)]
queries

['Technology Companies in Philadelphia, PA',
 'Technology Companies in Greater Philadelphia Region',
 'Technology Companies in Havertown, PA',
 'Technology Companies in Mainline PA',
 'Tech Companies in Philadelphia, PA',
 'Tech Companies in Greater Philadelphia Region',
 'Tech Companies in Havertown, PA',
 'Tech Companies in Mainline PA',
 'AI Companies in Philadelphia, PA',
 'AI Companies in Greater Philadelphia Region',
 'AI Companies in Havertown, PA',
 'AI Companies in Mainline PA',
 'Artificial Intelligence Companies in Philadelphia, PA',
 'Artificial Intelligence Companies in Greater Philadelphia Region',
 'Artificial Intelligence Companies in Havertown, PA',
 'Artificial Intelligence Companies in Mainline PA',
 'Machine Learning Companies in Philadelphia, PA',
 'Machine Learning Companies in Greater Philadelphia Region',
 'Machine Learning Companies in Havertown, PA',
 'Machine Learning Companies in Mainline PA',
 'ML Companies in Philadelphia, PA',
 'ML Companies in Greater Ph

In [107]:
all_dat = pd.DataFrame()

for query in queries:
    response = search_companies(query, api_key)

    df = pd.DataFrame(response.json().get('places', []))
    if df.shape[0] == 0:
        continue
    df['comp_name'] = df['displayName'].apply(lambda x: x['text'])
    df.drop('displayName', axis = 1, inplace = True)

    all_dat = pd.concat([all_dat, df], axis = 0, ignore_index = True)

In [108]:
all_dat.drop_duplicates(inplace = True)
all_dat

,formattedAddress,websiteUri,comp_name
0,"701 S 50th St Floor 2, Philadelphia, PA 19143,...",http://oceanringtech.com/contact/,Ocean Ring Technologies
1,"121 S Broad St floor 9, Philadelphia, PA 19107...",http://getguru.com/,"Guru Technologies, Inc."
2,"2400 Market St #200, Philadelphia, PA 19103, USA",http://arcweb.co/,Arcweb Technologies
3,"1515 Market St Suite 1200, Philadelphia, PA 19...",http://www.pumextech.com/,Pumex Technologies
4,"601 Walnut St 12th floor, Philadelphia, PA 191...",http://360.technology/,360 Technology LLC
...,...,...,...
919,"2005 Market St, Philadelphia, PA 19103, USA",https://www.leafnow.com/,"LEAF Commercial Capital, Inc."
923,"1845 Walnut St, Philadelphia, PA 19103, USA",https://bluebirdlending.com/,Bluebird Lending
960,"1100 E Hector St #210, Conshohocken, PA 19428,...",https://www.thrivestlink.com/,Thrivest Link Legal Funding™
965,"55 S Valley Rd Apt F3, Paoli, PA 19301, USA",https://bigfundings.com/,Big Fundings - Short term business loans


In [72]:
def find_linkedin_profile(company_name, api_key, cx_key):
    # Base URL for the Custom Search JSON API
    search_url = "https://www.googleapis.com/customsearch/v1"
    
    # Parameters for the API request
    params = {
        "q": f"{company_name}",  # Search query
        "cx": cx_key,  # Your Custom Search Engine ID
        "key": api_key,  # Your API key
    }
    
    # Make the API request
    response = requests.get(search_url, params=params)
    
    # Check if the request was successful
    if 'items' in response.json():
        return response.json()['items'][0]['link']
    else:
        return 0

In [109]:
li_prof = [find_linkedin_profile(company_name, api_key, cx_key) for company_name in list(all_dat['comp_name'])]

In [110]:
all_dat['linkedin_profile'] = li_prof

In [111]:
all_dat

,formattedAddress,websiteUri,comp_name,linkedin_profile
0,"701 S 50th St Floor 2, Philadelphia, PA 19143,...",http://oceanringtech.com/contact/,Ocean Ring Technologies,https://www.linkedin.com/company/ocean-ring-te...
1,"121 S Broad St floor 9, Philadelphia, PA 19107...",http://getguru.com/,"Guru Technologies, Inc.",https://www.linkedin.com/company/guru-technolo...
2,"2400 Market St #200, Philadelphia, PA 19103, USA",http://arcweb.co/,Arcweb Technologies,https://www.linkedin.com/company/arcweb
3,"1515 Market St Suite 1200, Philadelphia, PA 19...",http://www.pumextech.com/,Pumex Technologies,https://www.linkedin.com/company/pumex-computi...
4,"601 Walnut St 12th floor, Philadelphia, PA 191...",http://360.technology/,360 Technology LLC,https://www.linkedin.com/company/360technology...
...,...,...,...,...
919,"2005 Market St, Philadelphia, PA 19103, USA",https://www.leafnow.com/,"LEAF Commercial Capital, Inc.",0
923,"1845 Walnut St, Philadelphia, PA 19103, USA",https://bluebirdlending.com/,Bluebird Lending,0
960,"1100 E Hector St #210, Conshohocken, PA 19428,...",https://www.thrivestlink.com/,Thrivest Link Legal Funding™,0
965,"55 S Valley Rd Apt F3, Paoli, PA 19301, USA",https://bigfundings.com/,Big Fundings - Short term business loans,0
